In [8]:
# Check channel count of a WAV file
from scipy.io.wavfile import read as wav_read, write as wav_write

def get_channel_count(file_path):
    """
    Get the channel count of a WAV file.
    :param file_path: Path to the WAV file
    :return: Number of channels
    """
    # Load the audio file
    sample_rate, audio = wav_read(file_path)
    
    # Check the shape of the audio data
    if len(audio.shape) > 1:
        return audio.shape[1]  # Stereo or multi-channel
    else:
        return 1  # Mono

# Example usage
music_file_path = "Data/CTHS3_Acid_Bass_Loop_01_C_123.wav"
music_channel_count = get_channel_count(music_file_path)
print(f"Channel Count: {music_channel_count}")

Channel Count: 2


/var/folders/b0/m80gf4wx3pvdbp860pz1fqwc0000gn/T/ipykernel_57196/765551201.py:11: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sample_rate, audio = wav_read(file_path)


In [9]:
# Downmix to mono if stereo
sample_rate, music_audio = wav_read(music_file_path)
if music_channel_count > 1:
    # Assuming the first channel is the left and the second is the right
    music_audio_mono = music_audio.mean(axis=1)
    print("Downmixed to mono.")
else:
    music_audio_mono = music_audio
    print("Already mono.")
# Save the downmixed audio if needed
wav_write("Output/downmixed_music_mono.wav", sample_rate, music_audio_mono.astype(music_audio.dtype))

Downmixed to mono.


/var/folders/b0/m80gf4wx3pvdbp860pz1fqwc0000gn/T/ipykernel_57196/1076059340.py:2: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sample_rate, music_audio = wav_read(music_file_path)


In [14]:
# Upmix to stereo using interleaved samples if mono
import numpy as np

sine_file_path = "Data/sine_wave_44100Hz_16bit_1min.wav"
sine_sample_rate, sine_audio = wav_read(sine_file_path)
sine_channel_count = get_channel_count(sine_file_path)

if sine_channel_count == 1:
    # Ensure the mono signal has an even number of samples for interleaving
    if len(sine_audio) % 2 != 0:
        sine_audio = sine_audio[:-1]  # Drop the last sample if odd

    # Interleave samples for stereo upmixing
    interleaved = np.empty((len(sine_audio) * 2,), dtype=sine_audio.dtype)
    interleaved[0::2] = sine_audio  # Left channel gets even indices
    interleaved[1::2] = sine_audio  # Right channel gets odd indices

    # Reshape into a stereo signal
    sine_audio_stereo = interleaved.reshape(-1, 2)
    print("Upmixed to stereo using interleaved samples.")
elif sine_channel_count > 2:
    # If more than 2 channels, just take the first two channels
    sine_audio_stereo = sine_audio[:, :2]
    print("More than 2 channels, taking the first two.")
else:
    sine_audio_stereo = sine_audio
    print("Already stereo.")

# Save the upmixed audio if needed
wav_write("Output/upmixed_sine_stereo_interleaved_fixed2.wav", sine_sample_rate, sine_audio_stereo.astype(sine_audio.dtype))

# Check the channel count of the upmixed audio
sine_channel_count_upmixed = get_channel_count("Output/upmixed_sine_stereo_interleaved_fixed2.wav")
print(f"Upmixed Channel Count: {sine_channel_count_upmixed}")

Upmixed to stereo using interleaved samples.
Upmixed Channel Count: 2
